In [1]:
%%configure -f
{
    "conf":
    {
        "spark.driver.extraJavaOptions" : "--add-opens java.base/jdk.internal.loader=ALL-UNNAMED",
        "spark.jars.packages": "com.google.protobuf:protobuf-java:3.25.5"
    }
}

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, -1, Finished, Available, Finished)

In [3]:
# Import the required geoanalytics_fabric modules
# ESRI - FABRIC reference: https://developers.arcgis.com/geoanalytics-fabric/

import geoanalytics_fabric
from geoanalytics_fabric.sql import functions as ST
from geoanalytics_fabric import extensions
from geoanalytics_fabric.tools import Clip
from geoanalytics_fabric.tools import AggregatePoints


StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 4, Finished, Available, Finished)

In [4]:
# Get ROI based on BAG-woonplaats (=Loppersum)

# import required modules
from pyspark.sql.functions import col

# Read woonplaats data
path_woonplaats = "Files/BAG NL/BAG_woonplaats_202504.parquet"
df_woonplaats = spark.read.format("geoparquet").load(path_woonplaats)

# Filter the DataFrame where the "woonplaats" column contains the string "Loppersum"
df_loppersum = df_woonplaats.filter(col("woonplaats").contains("Loppersum"))

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 5, Finished, Available, Finished)

In [5]:
# Read building polygon data

path_building = "Files/BAG NL/BAG_pand_202504.parquet"
df_buildings = spark.read.format("geoparquet").load(path_building)

df_buildings.count()

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 6, Finished, Available, Finished)

11189134

In [6]:
# Select buildings for ROI

# Clip the BAG buildings to the gemeente Loppersum boundary
df_buildings_roi = Clip().run(input_dataframe=df_buildings,
                    clip_dataframe=df_loppersum)

# Count and show the top 5 results
df_buildings_roi.count()
#df_buildings_roi.show(5)

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 7, Finished, Available, Finished)

2492

In [7]:
# select only buildings older then AHN data (AHN2 (Groningen) = 2009) and with a status in use (Pand in gebruik)

df_buildings_roi_select = df_buildings_roi.where((df_buildings_roi.bouwjaar<2009) & (df_buildings_roi.status=='Pand in gebruik'))

df_buildings_roi_select.count()
#display(df_buildings_roi_select)

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 8, Finished, Available, Finished)

1196

In [ ]:
# write selected buildings

path_buildings_selected = "Files/Results/Buildings/Loppersum/buildings_older_2009.parquet"

df_buildings_roi_select.write.save(path_buildings_selected, format="geoparquet")

In [9]:
# Read AHN data
path_ahn2_parquet = "Files/AHN lidar/AHN2_parquet/ahn2.parquet"
path_ahn3_parquet = "Files/AHN lidar/AHN3_parquet/ahn3.parquet"
path_ahn4_parquet = "Files/AHN lidar/AHN4_parquet/ahn4.parquet"
df_ahn2 = spark.read.format("geoparquet").load(path_ahn2_parquet)
df_ahn3 = spark.read.format("geoparquet").load(path_ahn3_parquet)
df_ahn4 = spark.read.format("geoparquet").load(path_ahn4_parquet)	

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 10, Finished, Available, Finished)

In [10]:
# Get z value as a separate column - as input for statistics
df_ahn2 = df_ahn2.select('*', (ST.z("rd_point").alias("T1_z")))
df_ahn3 = df_ahn3.select('*', (ST.z("rd_point").alias("T2_z")))
df_ahn4 = df_ahn4.select('*', (ST.z("rd_point").alias("T3_z")))

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 11, Finished, Available, Finished)

In [45]:
# show number of lidar points per data set 
import pandas as pd
pandas_df = pd.DataFrame({
'T1': [df_ahn2.count()],
'T2': [df_ahn3.count()],
'T3': [df_ahn4.count()]
})

# Convert Pandas DataFrame to PySpark DataFrame
dfFromPandas = spark.createDataFrame(pandas_df)
dfFromPandas.show()


StatementMeta(, f57e61bd-993d-4b03-9f60-b15ed338d03e, 50, Finished, Available, Finished)

+---------+----------+----------+
|       T1|        T2|        T3|
+---------+----------+----------+
|107102284|1473917112|2184749356|
+---------+----------+----------+



In [11]:
# Select and aggregrate lidar points from buildings within ROI

df_ahn2_result = AggregatePoints() \
            .setPolygons(df_buildings_roi_select) \
            .addSummaryField(summary_field="T1_z", statistic="Mean", alias="T1_z_mean") \
            .addSummaryField(summary_field="T1_z", statistic="stddev", alias="T1_z_stddev") \
            .run(df_ahn2)

df_ahn3_result = AggregatePoints() \
            .setPolygons(df_buildings_roi_select) \
            .addSummaryField(summary_field="T2_z", statistic="Mean", alias="T2_z_mean") \
            .addSummaryField(summary_field="T2_z", statistic="stddev", alias="T2_z_stddev") \
            .run(df_ahn3)

df_ahn4_result = AggregatePoints() \
            .setPolygons(df_buildings_roi_select) \
            .addSummaryField(summary_field="T3_z", statistic="Mean", alias="T3_z_mean") \
            .addSummaryField(summary_field="T3_z", statistic="stddev", alias="T3_z_stddev") \
            .run(df_ahn4)


StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 12, Finished, Available, Finished)

In [12]:
results_ahn2_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn2.parquet"
results_ahn3_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn3.parquet"
results_ahn4_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn4.parquet"

df_ahn2_result.write.save(results_ahn2_path, format="geoparquet")
df_ahn3_result.write.save(results_ahn3_path, format="geoparquet")
df_ahn4_result.write.save(results_ahn4_path, format="geoparquet")

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 13, Finished, Available, Finished)

In [13]:
# Load geo-parquet file, with BAG polygonen
results_ahn2_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn2.parquet"
results_ahn3_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn3.parquet"
results_ahn4_path = "Files/Results/lidar/Loppersum/Building_aggregations/ahn4.parquet"

df_test = spark.read.format("geoparquet").load(results_ahn4_path)

df_test.printSchema()

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 14, Finished, Available, Finished)

root
 |-- feature_id: long (nullable = true)
 |-- rdf_seealso: string (nullable = true)
 |-- identificatie: string (nullable = true)
 |-- bouwjaar: long (nullable = true)
 |-- status: string (nullable = true)
 |-- gebruiksdoel: string (nullable = true)
 |-- oppervlakte_min: long (nullable = true)
 |-- oppervlakte_max: long (nullable = true)
 |-- aantal_verblijfsobjecten: long (nullable = true)
 |-- geometry: polygon (nullable = true)
 |-- clip_geometry: polygon (nullable = true)
 |-- COUNT: double (nullable = true)
 |-- T3_z_mean: double (nullable = true)
 |-- T3_z_stddev: double (nullable = true)



In [14]:
df_test.count()

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 15, Finished, Available, Finished)

1196

In [15]:
display(df_test)

StatementMeta(, 082e67e2-ee41-451f-af8a-1a5ca2136f29, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3754bf14-c9c2-4d6f-a4ec-d271aba9360e)